# Overall summary

This notebook provides an **overall summary** of the sharing of models against our best practices for enabling other to use and execute the simulation model.  In summary this is defined as:

## Data used in analysis

The dataset is a subset of the main review - limited to models shared.  The type of model shared is coded as **Visual Interactive Modelling (VIM)** based (e.g Anylogic, Simul8, Arena) versus **CODE** (e.g. Matlab, Python, SimPy, Java, R Simmer).

> The data can be found here: https://raw.githubusercontent.com/TomMonks/des_sharing_lit_review/main/data/bp_audit.zip

The following fields are analysed in this notebook.

* `model_format` - VIM or CODE
* `model_has_doi`  - do the model artefacts have their own minted DOI? (0/1)
* `orcid` - do the researchers provide an ORCID with the model? (0/1)
* `license` - does the model have an explicit license defining how it can be used? (str)
* `readme` - is there an obvious file(s) where a user would look first? (0/1)             
* `steps_run` - are there steps to run a model? (0/1)
* `formal_dep_mgt` - has the model been shared with formal software dependency management? (0/1)
* `informal_dep_mgt` - have any informal methods of dependency management been shared?  E.g. a list of software requirements. (0/1)
* `evidence_testing` - do the model and artefacts in the repository contain any evidence that they have been tested? (0/1)
* `downloadable` - can the model and artefacts be downloaded and executed locally? (0/1)
* `interactive_online` - can the model and its artefacts be executed online without local installation? (0/1)

## 1. Imports

### 1.1. Standard

In [1]:
import pandas as pd
import numpy as np

### 1.2 Preprocessing

In [2]:
from preprocessing import load_clean_bpa, drop_columns

## 2. Constants

In [3]:
FILE_NAME = 'https://raw.githubusercontent.com/TomMonks/' \
    + 'des_sharing_lit_review/main/data/bp_audit.zip'

## 3. Analysis functions

A number of simple functions to conduct the analysis and format output.

In [4]:
def balance_of_model_format(df):
    '''
    REturns the counts of VIM versus code
    
    Params:
    -------
    df: pd.DataFrame
        Subset of the best practice dataset to analyse
        
    Returns:
    (labels: List, counts: List)
    '''
    unique_elements, counts_elements = np.unique(df['model_format'], 
                                                 return_counts=True)
    return unique_elements, counts_elements

In [5]:
def category_frequencies_by_model_format(df, cols):
    '''
    Calculate the frequencies of 0/1s for VIM versus code.
    Return concatenated in a pandas dataframe
    
    Params:
    ------
    df: pd.DataFrame
        DAtaframe containing subset of best practice audit to summarise.
    
    Returns:
    -------
    pd.DataFrame
    '''

    # key to select fields where category is 1.
    key = [('CODE', 1), ('VIM', 1)]

    df = pd.DataFrame()

    # operation needs to be done separetly on each criteria then combined.
    for col in list(clean[cols]):
        # group by VIM and code and get frequencies of 1/0
        results = clean.groupby('model_format')[col].value_counts(dropna=False)
        # concat to single dataframe
        df = pd.concat([df, results.loc[key]], axis=1)
        
    # drop multi-index, transpose and relabel
    df = df.reset_index()
    df = df.T
    df = df.drop(['level_0', 'level_1'])
    df.columns = ['CODE', 'VIM']
    
    # add percentages
    # get total number of code and vim based models.
    _, (n_code, n_vim) = balance_of_model_format(clean)
    per_cols = ['CODE_%', 'VIM_%']
    df[per_cols[0]] = (df['CODE'] / n_code * 100).map('{:,.1f}'.format)
    df[per_cols[1]] = (df['VIM'] / n_vim * 100).map('{:,.1f}'.format)
    
    return df

In [6]:
def model_has_license(license):
    '''
    Recode the license column from multiple categories down to binary.
    None = 0 else 1
    
    Params:
    ------
    license: pd.Series
        The series containing the license info
        
    Returns:
    -------
    int
    '''
    if license == "None":
        return 0
    else:
        return 1

In [1]:
def format_bpa_results(summary):
    '''
    Convert 4 column able of n and % into two 
    columns where each column is n (%)
    
    Params:
    -------
    summary: pd.DataFrame
        The unformatted table.  Assumes 4 cols and index.
        
    Returns:
    -------
    pd.DataFrame
    '''
    
    row_headings = ['Model has DOI',
                    'ORCID',
                    'Licensed',
                    'Readme',
                    'Steps to run',
                    'Formal Dep Mgt',
                    'Informal Dep Mgt',
                    'Evidence of testing',
                    'Model downloadable',
                    'Model interactive online']
    
    summary['CODE (\%)'] = summary['CODE'].map('{:,.0f}'.format) \
        + ' (' + summary['CODE_%'] + ')'
    
    summary['VIM (\%)'] = summary['VIM'].map('{:,.0f}'.format) \
        + ' (' + summary['VIM_%'] + ')'
    
    
    summary = summary.drop(['CODE', 'VIM', 'CODE_%', 'VIM_%'], axis=1)
    summary['criteria'] = row_headings
    summary = summary.set_index('criteria')
    return summary

## 4. Load and inspect dataset

The clean data set has 27 fields included.  These are listed below.  

In [7]:
clean = load_clean_bpa(FILE_NAME)
clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39 entries, 0 to 38
Data columns (total 27 columns):
 #   Column                        Non-Null Count  Dtype   
---  ------                        --------------  -----   
 0   model_format                  39 non-null     category
 1   key                           39 non-null     object  
 2   item_type                     39 non-null     category
 3   pub_yr                        39 non-null     int64   
 4   author                        39 non-null     object  
 5   doi                           38 non-null     object  
 6   reporting_guidelines_mention  39 non-null     category
 7   covid                         39 non-null     category
 8   sim_software                  39 non-null     object  
 9   foss_sim                      39 non-null     category
 10  model_archive                 4 non-null      object  
 11  model_repo                    18 non-null     object  
 12  model_journal_supp            9 non-null      object

## 5. Results

### 5.1 Summary table.

The table illustrates the number of 

In [45]:
cols = ['model_has_doi', 'orcid', 'license_y', 'readme', 'steps_run', 
        'formal_dep_mgt', 'informal_dep_mgt', 
        'evidence_testing', 'downloadable', 'interactive_online']

clean['license_y'] = clean['license'].apply(model_has_license)
unformatted = category_frequencies_by_model_format(clean, cols)
unformatted

,CODE,VIM,CODE_%,VIM_%
model_has_doi,3,3,11.5,23.1
orcid,3,3,11.5,23.1
license_y,13,5,50.0,38.5
readme,18,6,69.2,46.2
steps_run,10,3,38.5,23.1
formal_dep_mgt,6,0,23.1,0.0
informal_dep_mgt,5,8,19.2,61.5
evidence_testing,3,0,11.5,0.0
downloadable,26,9,100.0,69.2
interactive_online,3,5,11.5,38.5


### 5.2 Formatted Results for paper + $\LaTeX$

In [63]:
table = format_bpa_results(unformatted)
table

,CODE (\%),VIM (\%)
criteria,,
Model has DOI,3 (11.5),3 (23.1)
ORCID,3 (11.5),3 (23.1)
Licensed,13 (50.0),5 (38.5)
Readme,18 (69.2),6 (46.2)
Steps to run,10 (38.5),3 (23.1)
Formal Dep Mgt,6 (23.1),0 (0.0)
Informal Dep Mgt,5 (19.2),8 (61.5)
Evidence of testing,3 (11.5),0 (0.0)
Model downloadable,26 (100.0),9 (69.2)


In [69]:
# output as latex
print(table.style.to_latex(hrules=True, 
                          label="Table:bpa_results", 
                          caption="Best practice audit results"))

\begin{table}
\caption{Best practice audit results}
\label{Table:bpa_results}
\begin{tabular}{lll}
\toprule
 & CODE (\%) & VIM (\%) \\
criteria &  &  \\
\midrule
Model has DOI & 3 (11.5) & 3 (23.1) \\
ORCID & 3 (11.5) & 3 (23.1) \\
Licensed & 13 (50.0) & 5 (38.5) \\
Readme & 18 (69.2) & 6 (46.2) \\
Steps to run & 10 (38.5) & 3 (23.1) \\
Formal Dep Mgt & 6 (23.1) & 0 (0.0) \\
Informal Dep Mgt & 5 (19.2) & 8 (61.5) \\
Evidence of testing & 3 (11.5) & 0 (0.0) \\
Model downloadable & 26 (100.0) & 9 (69.2) \\
Model interactive online & 3 (11.5) & 5 (38.5) \\
\bottomrule
\end{tabular}
\end{table}

